In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llm-detect-ai-generated-text/sample_submission.csv
/kaggle/input/llm-detect-ai-generated-text/train_prompts.csv
/kaggle/input/llm-detect-ai-generated-text/test_essays.csv
/kaggle/input/llm-detect-ai-generated-text/train_essays.csv
/kaggle/input/llm-generated-essays/ai_generated_train_essays.csv
/kaggle/input/llm-generated-essays/ai_generated_train_essays_gpt-4.csv
/kaggle/input/daigt-v2-train-dataset/train_v2_drcat_02.csv


In [2]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import SGDClassifier

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
def read_data(training_path, test_path):
    train = pd.read_csv(training_path)
    train = train[train.RDizzl3_seven == True].reset_index(drop=True)
    test = pd.read_csv(test_path)
    df = pd.concat([train['text'], test['text']], axis=0)
    
    return df, train, test

In [4]:
def vectorization(data_frame):
    vectorizer = TfidfVectorizer(ngram_range=(1, 3),sublinear_tf=True)
    X = vectorizer(data_frame)
    return X

In [5]:
def model_prediction(train, X):
    lr_model = LogisticRegression()
    sgd_model = SGDClassifier(max_iter=5000, tol=1e-3, loss="modified_huber")
    ensemble = VotingClassifier(estimators=[('lr', lr_model), ('sgd', sgd_model)], weights=[0.1,0.9], voting='soft')
    ensemble.fit(X[:train.shape[0]], train.label)
    preds_test = ensemble.predict_proba(X[train.shape[0]:])[:,1]
    return preds_test

In [6]:
training_path = '/kaggle/input/daigt-v2-train-dataset/train_v2_drcat_02.csv'
test_path = '/kaggle/input/llm-detect-ai-generated-text/test_essays.csv'
data_frame, train, test = read_data(training_path, test_path)

In [7]:
train = pd.read_csv("/kaggle/input/daigt-v2-train-dataset/train_v2_drcat_02.csv")
train = train[train.RDizzl3_seven == True].reset_index(drop=True)
test = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/test_essays.csv')
df = pd.concat([train['text'], test['text']], axis=0)
vectorizer = TfidfVectorizer(ngram_range=(1, 3),sublinear_tf=True)
X = vectorizer.fit_transform(df)

In [8]:
predict = model_prediction(train, X)

/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtyp

In [9]:
pd.DataFrame({'id':test["id"], 'generated':predict}).to_csv('submission.csv', index=False)